In [1]:
!pip install pandas scikit-learn scipy numpy

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 67.4 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [2]:
import mesa
print(mesa.__version__)

3.1.3


In [5]:
from enum import Enum

# Enum to track the status of customer orders
class OrderStatus(Enum):
    WAITING = 0    # Customer hasn't ordered yet
    ORDERED = 1    # Order has been placed
    PREPARING = 2  # Food is being prepared
    SERVED = 3     # Food has been delivered

# Available food options in the restaurant
food_options = ["vegetarian", "meat", "gluten free"]

In [6]:
import mesa
import random

class CustomerAgent(mesa.Agent):
    def __init__(self):
        super().__init__()
        # Initialize customer properties
        self.food_preference = random.choice(food_options)
        self.waiting_time = 0                         # Time spent waiting
        self.order_status = OrderStatus.WAITING       # Current order status
        self.satisfaction = 0                         # Overall satisfaction (0-100)
        self.tip = 0                                 # Amount of tip given
        self.rating = 0                              # Rating given (1-5)
        self.assigned_waiter = None                  # Reference to assigned waiter

    def step(self):
        # Increment waiting time if not served yet
        if self.order_status != OrderStatus.SERVED:
            self.waiting_time += 1

    def order_dish(self, waiter):
        # Place order with waiter if not already ordered
        if self.order_status == OrderStatus.WAITING:
            self.order_status = OrderStatus.ORDERED
            self.assigned_waiter = waiter
            return self.food_preference

    def rate_experience(self):
        # Calculate satisfaction based on waiting time
        self.satisfaction = max(0, 100 - self.waiting_time)
        # Convert satisfaction to 1-5 star rating
        self.rating = max(1, self.satisfaction / 20)
        # Calculate tip based on satisfaction
        self.tip = self.satisfaction / 10



In [7]:
class WaiterAgent(mesa.Agent):
    def __init__(self):
        super().__init__()
        # Initialize waiter properties
        self.busy = False                # Current serving status
        self.current_orders = []         # List of (customer, order) tuples
        self.tips = 0                    # Total tips received
        self.avg_rating = 0              # Average rating from customers
        self.ratings_count = 0           # Number of ratings received
        self.served_customers = 0        # Total customers served

    def take_order(self, customer):
        # Take order from customer if waiter is available
        if not self.busy:
            order = customer.order_dish(self)
            if order:
                self.current_orders.append((customer, order))
                self.busy = True
                return True
        return False

    def serve_dish(self, customer):
        # Serve food to customer and collect feedback
        if customer in [c for c, _ in self.current_orders]:
            customer.order_status = OrderStatus.SERVED
            # Remove served customer from current orders
            self.current_orders = [(c, o) for c, o in self.current_orders if c != customer]
            self.busy = len(self.current_orders) > 0
            self.served_customers += 1

            # Update waiter's performance metrics
            rating = customer.rating
            tip = customer.tip
            self.tips += tip
            self.ratings_count += 1
            self.avg_rating = ((self.avg_rating * (self.ratings_count - 1)) + rating) / self.ratings_count

In [8]:
class ManagerAgent(mesa.Agent):
    def __init__(self):
        super().__init__()
        # Initialize manager properties
        self.food_inventory = {option: 100 for option in food_options}  # Initial stock
        # Track daily statistics
        self.daily_stats = {
            'total_customers': 0,        # Total customers in restaurant
            'avg_waiting_time': 0,       # Average customer waiting time
            'active_waiters': 0,         # Number of available waiters
            'profit': 0                  # Daily profit
        }

    def step(self):
        # Update daily statistics
        model = self.model
        self.daily_stats['total_customers'] = len(model.customers)
        self.daily_stats['active_waiters'] = len([w for w in model.waiters if not w.busy])
        self.daily_stats['avg_waiting_time'] = np.mean([c.waiting_time for c in model.customers])

    def order_food(self, food_type, amount):
        # Replenish food inventory
        self.food_inventory[food_type] += amount

    def calculate_profit(self):
        # Calculate daily profit considering various costs
        total_sales = sum(w.tips for w in self.model.waiters)
        staff_costs = len(self.model.waiters) * 10  # Fixed cost per waiter
        food_costs = sum(100 - amount for amount in self.food_inventory.values())
        self.daily_stats['profit'] = total_sales - (staff_costs + food_costs)

In [9]:
import numpy as np

class RestaurantModel(mesa.Model):
    def __init__(self, n_customers, n_waiters, seed=None):
        super().__init__(seed=seed)

        # Create agents
        self.customers = [CustomerAgent() for _ in range(n_customers)]
        self.waiters = [WaiterAgent() for _ in range(n_waiters)]
        self.manager = ManagerAgent()
        self.agents = self.customers + self.waiters + [self.manager]

        # Set up data collection for model metrics
        self.datacollector = mesa.DataCollector(
            model_reporters={
                "Customer_Count": lambda m: len(m.customers),
                "Average_Wait_Time": lambda m: np.mean([c.waiting_time for c in m.customers]),
                "Customer_Satisfaction": lambda m: np.mean([c.satisfaction for c in m.customers]),
                "Profit": lambda m: m.manager.daily_stats['profit']
            }
        )

    def step(self):
        # Collect data and execute agent steps in random order
        self.datacollector.collect(self)
        random.shuffle(self.agents)
        for agent in self.agents:
            agent.step()